In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.v2 as transforms
import torchattacks
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]


def load_data():
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )

    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    testset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform
    )

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=False)
    # os.makedirs("./images/train", exist_ok=True)
    # for i, (inputs, labels) in enumerate(trainloader):
    #     for j in range(inputs.size(0)):
    #         torchvision.utils.save_image(
    #             inputs[j], f"./images/train/{i*128+j}_label_{labels[j]}.png"
    #         )
    testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)
    # os.makedirs("./images/valid", exist_ok=True)
    # for i, (inputs, labels) in enumerate(testloader):
    #     for j in range(inputs.size(0)):
    #         torchvision.utils.save_image(
    #             inputs[j], f"./images/valid/{i*128+j}_label_{labels[j]}.png"
    #         )
    return trainloader, testloader


def evaluate(model, dataloader, mode="原始"):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"{mode}数据集准确率: {acc:.2f}%")
    return acc


def load_images_from_folder(folder, defends=None):
    images = []
    labels = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".png"):
            img_path = os.path.join(folder, filename)
            label = int(filename.split("_label_")[1].split(".")[0])

            img = Image.open(img_path)
            if defends == "blur":
                img = transforms.GaussianBlur(3)(img)
            elif defends == "jpeg":
                img = transforms.JPEG(80)(img)
            transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize(
                        (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
                    ),
                ]
            )
            img = transform(img)
            images.append(img)
            labels.append(label)

    return torch.stack(images), torch.tensor(labels)


def save_raw_images(images, labels, folder):
    """直接保存原始像素值，不进行标准化处理"""
    os.makedirs(folder, exist_ok=True)

    for i, (img, label) in enumerate(zip(images, labels)):
        # 转到CPU并检查是否需要反标准化
        img = img.cpu().detach()

        if hasattr(img, "requires_grad"):
            # 标准化参数
            mean = torch.tensor(MEAN).view(3, 1, 1)
            std = torch.tensor(STD).view(3, 1, 1)

            # 反标准化获取原始像素值
            raw_img = img * std + mean
        else:
            raw_img = img

        # 转换为uint8格式
        raw_img = (raw_img.permute(1, 2, 0) * 255).numpy()
        raw_img = np.clip(raw_img, 0, 255).astype(np.uint8)

        # 使用PNG格式无损保存
        img_path = os.path.join(folder, f"{i}_label_{label}.png")
        Image.fromarray(raw_img).save(img_path, format="PNG")


def generate_adv_samples(model, testloader, path):
    """生成对抗样本并保存"""
    print(f"攻击前准确率: {evaluate(model, testloader):.2f}%")

    # 设置保存路径
    adv_path = path
    os.makedirs(adv_path, exist_ok=True)

    # 初始化PGD攻击
    atk = torchattacks.PGD(model, eps=8 / 255, alpha=2 / 255, steps=10)
    atk.set_normalization_used(mean=MEAN, std=STD)

    adv_images, adv_labels = [], []
    total_batches = len(testloader)

    for batch_idx, (images, labels) in enumerate(testloader):
        # 将整个batch移至GPU并攻击
        images, labels = images.to(device), labels.to(device)
        adv_batch = atk(images, labels)

        # 收集结果
        adv_images.append(adv_batch.cpu())
        adv_labels.append(labels.cpu())

        print(f"\r处理进度: [{batch_idx+1}/{total_batches}]", end="")

    # 合并所有batch的结果
    adv_images = torch.cat(adv_images, 0)
    adv_labels = torch.cat(adv_labels, 0)

    # 保存对抗图像
    save_raw_images(adv_images, adv_labels, adv_path)

    # 创建对抗样本数据集
    adv_dataset = torch.utils.data.TensorDataset(adv_images, adv_labels)
    att_testloader = torch.utils.data.DataLoader(
        adv_dataset, batch_size=testloader.batch_size, shuffle=False
    )

    print(f"\n攻击后准确率: {evaluate(model, att_testloader, mode='对抗'):.2f}%")
    print(f"对抗图像已保存至: {adv_path}")

    return att_testloader

In [7]:
from mair.hub import load_pretrained
# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_pretrained("CIFAR10_ResNet18_Standard", flag='Last', save_dir="./")
model.cuda()

trainloader, testloader = load_data()

generate_adv_samples(model, testloader, path="./images/valid_adv")
generate_adv_samples(model, trainloader, path="./images/train_adv")

CIFAR10_ResNet18_Standard
Method         : Standard
Architecture   : ResNet18
Batch Size     : 128
Aug.           : True
AWP            : False
Extra Data     : False
Clean(Last)    : 85.08%
PGD(Last)      : 49.34%
Clean(Best)    : 84.35%
PGD(Best)      : 56.67%
ResNet18 is loaded.


/home/shenss/miniconda3/envs/py311cu124/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
原始数据集准确率: 95.40%
攻击前准确率: 95.40%
处理进度: [79/79]对抗数据集准确率: 0.04%

攻击后准确率: 0.04%
对抗图像已保存至: ./images/valid_adv
原始数据集准确率: 100.00%
攻击前准确率: 100.00%
处理进度: [391/391]对抗数据集准确率: 0.06%

攻击后准确率: 0.06%
对抗图像已保存至: ./images/train_adv


In [ ]:
# 加载保存的图像
print("\n加载保存的图像...")
orig_images, orig_labels = load_images_from_folder('images/valid')
adv_images, adv_labels = load_images_from_folder('images/valid_adv')

orig_images_jpeg, orig_labels_jpeg = load_images_from_folder('images/valid', defends='jpeg')
adv_images_jpeg, adv_labels_jpeg = load_images_from_folder('images/valid_adv', defends='jpeg')

orig_images_gs, orig_labels_gs = load_images_from_folder('images/valid', defends='blur')
adv_images_gs, adv_labels_gs = load_images_from_folder('images/valid_adv', defends='blur')

orig_images_ht, orig_labels_ht = load_images_from_folder('images/valid_pure')
adv_images_ht, adv_labels_ht = load_images_from_folder('images/valid_adv_pure')

# 创建数据加载器
orig_dataset = torch.utils.data.TensorDataset(orig_images, orig_labels)
adv_dataset = torch.utils.data.TensorDataset(adv_images, adv_labels)

ori_jpeg_dataset = torch.utils.data.TensorDataset(orig_images_jpeg, orig_labels_jpeg)
adv_jpeg_dataset = torch.utils.data.TensorDataset(adv_images_jpeg, adv_labels_jpeg)

adv_gs_dataset = torch.utils.data.TensorDataset(adv_images_gs, adv_labels_gs)
ori_gs_dataset = torch.utils.data.TensorDataset(orig_images_gs, orig_labels_gs)

adv_ht_dataset = torch.utils.data.TensorDataset(adv_images_ht, adv_labels_ht)
ori_ht_dataset = torch.utils.data.TensorDataset(orig_images_ht, orig_labels_ht)

orig_loader = torch.utils.data.DataLoader(orig_dataset, batch_size=128, shuffle=False)
adv_loader = torch.utils.data.DataLoader(adv_dataset, batch_size=128, shuffle=False)

ori_jpeg_loader = torch.utils.data.DataLoader(ori_jpeg_dataset, batch_size=128, shuffle=False)
adv_jpeg_loader = torch.utils.data.DataLoader(adv_jpeg_dataset, batch_size=128, shuffle=False)

adv_gs_loader = torch.utils.data.DataLoader(adv_gs_dataset, batch_size=128, shuffle=False)
ori_gs_loader = torch.utils.data.DataLoader(ori_gs_dataset, batch_size=128, shuffle=False)

adv_ht_loader = torch.utils.data.DataLoader(adv_ht_dataset, batch_size=128, shuffle=False)
ori_ht_loader = torch.utils.data.DataLoader(ori_ht_dataset, batch_size=128, shuffle=False)

# 评估准确率
print("\n评估保存后的图像...")
#orig_acc = evaluate(model, orig_loader, mode='原始')
orig_acc = model.eval_accuracy(orig_loader)
print(f"原始准确率: {orig_acc:.2f}%")
orig_acc = model.eval_accuracy(adv_loader)
print(f"对抗准确率: {orig_acc:.2f}%")
# adv_acc = evaluate(model, adv_loader, mode='对抗')

# ori_gs_acc = evaluate(model, ori_gs_loader, mode='GS3原始')
ori_gs_acc = model.eval_accuracy(ori_gs_loader)
print(f"GS3原始准确率: {ori_gs_acc:.2f}%")
# adv_gs_acc = evaluate(model, adv_gs_loader, mode='GS3对抗')
adv_gs_acc = model.eval_accuracy(adv_gs_loader)
print(f"GS3对抗准确率: {adv_gs_acc:.2f}%")

# ori_jpeg_acc = evaluate(model, ori_jpeg_loader, mode='JPEG80原始')
ori_jpeg_acc = model.eval_accuracy(ori_jpeg_loader)
print(f"JPEG80原始准确率: {ori_jpeg_acc:.2f}%")
# adv_jpeg_acc = evaluate(model, adv_jpeg_loader, mode='JPEG80对抗')
adv_jpeg_acc = model.eval_accuracy(adv_jpeg_loader)
print(f"JPEG80对抗准确率: {adv_jpeg_acc:.2f}%")
# # ori_ht_acc = evaluate(model, ori_ht_loader, mode='HT原始')
ori_ht_acc = model.eval_accuracy(ori_ht_loader)
print(f"HT原始准确率: {ori_ht_acc:.2f}%")
# adv_ht_acc = evaluate(model, adv_ht_loader, mode='HT对抗')
adv_ht_acc = model.eval_accuracy(adv_ht_loader)
print(f"HT对抗准确率: {adv_ht_acc:.2f}%")


加载保存的图像...

评估保存后的图像...
原始准确率: 95.40%
对抗准确率: 0.04%
GS3原始准确率: 67.37%
GS3对抗准确率: 29.08%
JPEG80原始准确率: 87.09%
JPEG80对抗准确率: 49.54%
